In [ ]:
!pip install -U git+https://github.com/faustomorales/keras-ocr.git
!pip install imgaug
!pip install -U opencv-python

!gdown --id 1BEOWSoIbzXgPnxtaKdCuP9qxYuOgCwSd
!gdown --id 12JOXOOHnK8LIbzwxKHTZsLVyZVAGs2kE
!gdown --id 1dhn30gRDRKWiF24Gi8dd4bhEMxvVXi3_
!gdown --id 1Yq2LgCgstCSD9S1Hvz2y5EviucbIOX9o
%tensorflow_version 2.x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/faustomorales/keras-ocr.git to /tmp/pip-req-build-65nu9cio
  Running command git clone -q https://github.com/faustomorales/keras-ocr.git /tmp/pip-req-build-65nu9cio
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or ha

In [ ]:
import os
import math
import imgaug
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection
import tensorflow as tf
import zipfile
import typing

import keras_ocr

dir = "."

In [ ]:
def extract_train_dataset(main_dir="."):
  if not os.path.isdir(main_dir):
    os.mkdir(main_dir)
  train_image_dir = os.path.join(main_dir, "train_images")
  if not os.path.isdir(train_image_dir):
    os.mkdir(train_image_dir)
  training_zip_images_path = os.path.join(".", "Challenge2_Training_Task12_Images.zip")
  if len(os.listdir(train_image_dir)) != 229:
    with zipfile.ZipFile(training_zip_images_path, 'r') as zip_ref:
      zip_ref.extractall(train_image_dir)

  train_gt_dir = os.path.join(main_dir, "train_gt")
  if not os.path.isdir(train_gt_dir):
    os.mkdir(train_gt_dir)
  training_zip_gt_path = os.path.join(".", "Challenge2_Training_Task1_GT.zip")
  if len(os.listdir(train_gt_dir)) != 229:
      with zipfile.ZipFile(training_zip_gt_path) as zfile:
          zfile.extractall(train_gt_dir)

In [ ]:
def extract_test_dataset(main_dir="."):
  if not os.path.isdir(main_dir):
    os.mkdir(main_dir)
  test_image_dir = os.path.join(main_dir, "test_images")
  if not os.path.isdir(test_image_dir):
    os.mkdir(test_image_dir)
  test_zip_images_path = os.path.join(".", "Challenge2_Test_Task12_Images.zip")
  if len(os.listdir(test_image_dir)) != 233:
    with zipfile.ZipFile(test_zip_images_path, 'r') as zip_ref:
      zip_ref.extractall(test_image_dir)

  test_gt_dir = os.path.join(main_dir, "test_gt")
  if not os.path.isdir(test_gt_dir):
    os.mkdir(test_gt_dir)
  test_zip_gt_path = os.path.join(".", "Challenge2_Test_Task1_GT.zip")
  if len(os.listdir(test_gt_dir)) != 233:
      with zipfile.ZipFile(test_zip_gt_path) as zfile:
          zfile.extractall(test_gt_dir)

In [ ]:
def preprocess_train_dataset(dir="."):
  main_dir = os.path.join(dir, "icdar2013")
  extract_train_dataset(main_dir)
  dataset = []

  train_gt_dir = os.path.join(main_dir, "train_gt")
  train_image_dir = os.path.join(main_dir, "train_images")

  for filename in os.listdir(train_gt_dir):
    file_path = os.path.join(train_gt_dir, filename)
    image_id = filename.split("_")[1].split(".")[0]
    image_path = os.path.join(train_image_dir, image_id + ".jpg")
    lines = []
    with open(file_path, "r", encoding="utf8") as f:
      current_line: typing.List[typing.Tuple[np.ndarray, str]] = []
      for row in f.read().split("\n"):
        if row == "":
          lines.append(current_line)
          current_line = []
        else:
          line_row = row.split(" ")
          text = line_row[-1].strip("\"")
          x1, y1, x2, y2 = map(int, line_row[:4])
          current_line.append(
              (np.array([[x1, y1], [x2, y1], [x2, y2], [x1, y2]]), text)
          )
      lines = [line for line in lines if line]
      dataset.append((image_path, lines, 1)) 

  return dataset

In [ ]:
def preprocess_test_dataset(dir="."):
  main_dir = os.path.join(dir, "icdar2013")
  extract_test_dataset(main_dir)
  dataset = []

  test_gt_dir = os.path.join(main_dir, "test_gt")
  test_image_dir = os.path.join(main_dir, "test_images")

  for filename in os.listdir(test_gt_dir):
    file_path = os.path.join(test_gt_dir, filename)
    image_id = filename.split("_")[2].split(".")[0]
    image_path = os.path.join(test_image_dir, "img_" + image_id + ".jpg")
    lines = []
    with open(file_path, "r", encoding="utf8") as f:
      current_line: typing.List[typing.Tuple[np.ndarray, str]] = []
      for row in f.read().split("\n"):
        if row == "":
          lines.append(current_line)
          current_line = []
        else:
          line_row = row.split(", ")
          text = line_row[-1].strip("\"")
          x1, y1, x2, y2 = map(int, line_row[:4])
          current_line.append(
              (np.array([[x1, y1], [x2, y1], [x2, y2], [x1, y2]]), text)
          )
      lines = [line for line in lines if line]
      dataset.append((image_path, lines, 1)) 

  return dataset

In [ ]:
train_labels = preprocess_train_dataset(dir)
test_labels = preprocess_test_dataset(dir)
train_labels.extend(test_labels)

In [ ]:
print(len(train_labels))

462


In [ ]:
learning_rate = 0.0000005

detector = keras_ocr.detection.Detector()
detector.model.compile(
    loss="mse", 
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
    metrics=["acc"]
)

Looking for /root/.keras-ocr/craft_mlt_25k.h5


In [ ]:
detector.model.summary()

Model: "model_35"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 basenet.slice1.0 (Conv2D)      (None, None, None,   1792        ['input_18[0][0]']               
                                64)                                                               
                                                                                                  
 basenet.slice1.1 (BatchNormali  (None, None, None,   256        ['basenet.slice1.0[0][0]']       
 zation)                        64)                                                        

In [ ]:
def make_generators(train_labels, validation_labels, augmenter):
  generator_kwargs = {'width': 640, 'height': 640}
  training_image_generator = keras_ocr.datasets.get_detector_image_generator(
      labels=train_labels,
      augmenter=augmenter,
      **generator_kwargs
  )
  validation_image_generator = keras_ocr.datasets.get_detector_image_generator(
      labels=validation_labels,
      **generator_kwargs
  )
  training_generator, validation_generator = [
    detector.get_batch_generator(
        image_generator=image_generator, batch_size=batch_size
    ) for image_generator in
    [training_image_generator, validation_image_generator]
  ]
  return training_generator, validation_generator

In [ ]:
def train_detector_kfold(training_labels, augmenter, batch_size, callbacks=None, k=10, epochs=100):
  all_scores = []
  num_val_samples = len(training_labels) // k
  for i in range(k):
    print("Processing fold #", i+1)
    val = training_labels[i * num_val_samples: (i + 1) * num_val_samples]
    train = training_labels[:i * num_val_samples]
    train.extend(
        training_labels[(i + 1) * num_val_samples:]
    )
    training_gen, validation_gen = make_generators(
        train, val, augmenter
    )
    history = detector.model.fit(
          x=training_gen,
          steps_per_epoch=math.ceil(len(train) / batch_size),
          callbacks=callbacks,
          epochs=epochs,
    )
    val_loss, val_acc = detector.model.evaluate(
        x=validation_gen,
        steps=math.ceil(len(val) / batch_size),
    )
    all_scores.append(val_loss)
  val_loss_mean = np.mean(all_scores)
  return val_loss_mean

In [ ]:
batch_size = 1

augmenter = imgaug.augmenters.Sequential([
    imgaug.augmenters.Affine(
      scale=(1.0, 1.2),
      rotate=(-5, 5)
    ),
    imgaug.augmenters.GaussianBlur(sigma=(0, 0.5)),
    imgaug.augmenters.Multiply((0.8, 1.2), per_channel=0.2)
])

val_loss = train_detector_kfold(
    train_labels,
    augmenter,
    batch_size,
    k=4,
    epochs=3
)

Processing fold # 1
Epoch 1/3
347/347 [==============================] - 92s 255ms/step - loss: 0.0111 - acc: 0.9696
Epoch 2/3
347/347 [==============================] - 86s 247ms/step - loss: 0.0109 - acc: 0.9644
Epoch 3/3
115/115 [==============================] - 12s 96ms/step - loss: 0.0078 - acc: 0.9737
Processing fold # 2
Epoch 1/3
347/347 [==============================] - 85s 245ms/step - loss: 0.0100 - acc: 0.9658
Epoch 2/3
347/347 [==============================] - 85s 244ms/step - loss: 0.0091 - acc: 0.9693
Epoch 3/3
115/115 [==============================] - 12s 100ms/step - loss: 0.0057 - acc: 0.9687
Processing fold # 3
Epoch 1/3
347/347 [==============================] - 86s 249ms/step - loss: 0.0081 - acc: 0.9762
Epoch 2/3
347/347 [==============================] - 87s 250ms/step - loss: 0.0070 - acc: 0.9814
Epoch 3/3
115/115 [==============================] - 11s 92ms/step - loss: 0.0062 - acc: 0.9373
Processing fold # 4
Epoch 1/3
347/347 [==============================

In [ ]:
detector.model.save("detector_icdar2013.h5")

In [ ]:
from google.colab import files
files.download('/content/detector_icdar2013.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>